In [ ]:
!pip install docling ollama pymongo

In [ ]:
%%time
from docling.document_converter import DocumentConverter

source = "RUTA_ARCHIVO"
converter = DocumentConverter()
result = converter.convert(source)
resultado_markdown = result.document.export_to_markdown()
print(resultado_markdown)

## Limpieza de datos y registro en MongoDB

In [ ]:
import re

def eliminar_bloques_identificacion_paciente(texto):
    lineas = texto.splitlines()
    i = 0

    while i < len(lineas):
        if "Financiador:" in lineas[i]:
            # Ver si hay <!-- image --> 1 o 2 líneas arriba
            inicio_bloque = i
            if i >= 1 and '<!-- image -->' in lineas[i - 1]:
                inicio_bloque = i - 1
            elif i >= 2 and '<!-- image -->' in lineas[i - 2]:
                inicio_bloque = i - 2

            # Buscar el fin del bloque: ## IDENTIFICACIÓN DEL PACIENTE
            for j in range(i + 1, len(lineas)):
                if '## IDENTIFICACIÓN DEL PACIENTE' in lineas[j]:
                    # Eliminar desde inicio_bloque hasta j (inclusive)
                    del lineas[inicio_bloque:j + 1]
                    i = inicio_bloque - 1  # retroceder para continuar correctamente
                    break
        i += 1

    return '\n'.join(lineas)

# Aplicar
texto_limpio = eliminar_bloques_identificacion_paciente(resultado_markdown)


In [ ]:
# Elimina las nuevas apariciones del titulo HISTORICO DE ATENCIONES para que en una misma seccion figuren las tablas leidas
# Se intentó concatenar las tablas para tener una tabla unificada, pero en las pruebas se observó que la segunda tabla del
# historico de atencedentes no tenia la columna Remitido, lo cual no es grave, porq en el pdf no habia valor para las filas de esa column, sin embargo,
# al querer concatenarlas la cantidad de columnas no coincidió


def conservar_solo_primer_titulo_limpio(texto, titulo="## HISTÓRICO DE ATENCIONES"):
    # Buscar primera aparición exacta (con o sin espacios antes/después)
    match = re.search(rf'\s*{re.escape(titulo)}\s*', texto)
    if not match:
        return texto  # No se encontró

    # Dividir en dos partes: antes y después del primer match
    inicio = match.start()
    fin = match.end()

    antes = texto[:inicio]
    primera_aparicion = texto[inicio:fin]
    despues = texto[fin:]

    # Eliminar todas las demás apariciones del título (con espacios extra)
    despues_limpio = re.sub(rf'\s*{re.escape(titulo)}\s*', '', despues)

    return (antes + primera_aparicion + despues_limpio).strip()


texto_limpio = conservar_solo_primer_titulo_limpio(texto_limpio)

In [ ]:
# Elimnar comentario de fecha y hora de impresion del informe
def eliminar_fecha_hora_impreso(texto):
    return re.sub(
        r'\d{2}:\d{2}:\d{2}\s+\d{2}/\d{2}/\d{4}\s+Documento impreso al día',
        '',
        texto
    )

texto_limpio = eliminar_fecha_hora_impreso(texto_limpio)

In [ ]:
# Eliminar tags de imagen
texto_limpio = texto_limpio.replace("<!-- image -->", "")

# Eliminar cabecera INDICE que se repite muchas veces
texto_limpio = texto_limpio.replace("## ÍNDICE", "")

# Eliminar frase Firmado electronicamente
texto_limpio = texto_limpio.replace("Firmado electrónicamente", "")

# Eliminar líneas como "Documento impreso al día DD/MM/AAAA HH:MM:SS"
texto_limpio = re.sub(r'Documento impreso al día \d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}', '', texto_limpio)

# Eliminar líneas como "Página X de Y" (con cualquier número)
texto_limpio = re.sub(r'Página  \d+  de  \d+', '', texto_limpio)


In [ ]:
# Se quita los titulos NOTAS MEDICAS que se repiten
import re

def limpiar_titulos_notas_medicas(texto):
    lineas = texto.splitlines()
    i = 0
    bloques = []

    while i < len(lineas):
        if lineas[i].strip() == '## NOTAS MÉDICAS':
            inicio = i
            i += 1

            while i < len(lineas):
                if lineas[i].strip() == '## NOTAS MÉDICAS':
                    ventana = lineas[i+1:i+16]

                    # Verificamos si hay otro encabezado dentro de la ventana
                    hay_otro_titulo = any(l.strip() == '## NOTAS MÉDICAS' for l in ventana)
                    hay_fecha = any(re.search(r'Fecha:\s*\d{2}/\d{2}/\d{4}\s*\d{2}:\d{2}', l) for l in ventana)

                    if hay_otro_titulo:
                        # No considerar este como delimitador, seguir buscando
                        i += 1
                        continue

                    if hay_fecha:
                        # Este sí delimita el bloque
                        fin = i
                        break
                i += 1
            else:
                fin = len(lineas)

            bloque = '\n'.join(lineas[inicio:fin])
            bloques.append((inicio, fin, bloque))
            i = fin
        else:
            i += 1

    # Limpiar cada bloque: dejar solo el primer título
    for inicio, fin, bloque in reversed(bloques):
        partes = bloque.split('## NOTAS MÉDICAS')
        bloque_limpio = '## NOTAS MÉDICAS\n' + ''.join(partes[1:]).strip()
        lineas[inicio:fin] = bloque_limpio.splitlines()

    return '\n'.join(lineas)

texto_limpio = limpiar_titulos_notas_medicas(texto_limpio)

In [ ]:
print(texto_limpio)

## Extraer informacion del paciente

In [ ]:
import re

def extraer_y_eliminar_info_general(texto):
    # Buscar el bloque que comienza con INFORMACIÓN GENERAL VIGENTE y termina antes del siguiente encabezado ##
    match = re.search(r'## INFORMACIÓN GENERAL VIGENTE(.*?)(?=##|\Z)', texto, re.DOTALL)

    if not match:
        return None, texto  # No se encontró ese bloque

    contenido = match.group(1).strip()  # Contenido del bloque (sin el encabezado)

    # Quitar el bloque completo (incluyendo el título)
    texto_sin_bloque = texto.replace(match.group(0), '').strip()

    return contenido, texto_sin_bloque

info_general, texto_limpio = extraer_y_eliminar_info_general(texto_limpio)

In [2]:
# Estableciendo conexion con base de datos para guardar la informacion del paciente
from pymongo import MongoClient
from getpass import getpass


# Conectarse al servidor local (por defecto en el puerto 27017)
client = MongoClient(getpass("Ingresa cadena de conexion: "))

# Nombre de la bd
db = client["clinical-data"]

# Coleccion de pacientes
coleccion_pacientes = db["patients"]

# Coleccion de HC
coleccion_hc = db["clinical-history"]


In [ ]:
# Revisar informacion general extraida
print(info_general)

In [ ]:
#Obteniendo informacion de paciente con ayuda de llm para estructurar datos
from ollama import Client
from pydantic import BaseModel

client = Client(host="http://localhost:11434")

class Patient(BaseModel):
  tipo_identificacion: str
  numero_identificacion: str
  estado_civil: str
  telefono: str
  fecha_nacimiento: str
  nombre_completo: str
  genero: str
  ocupacion: str
  direccion: str
  lugar_residencia: str

response = client.chat(
  messages=[
    {
      'role': 'user',
      'content': f"Extrae la informacion de este paciente: \n {info_general}",
    }
  ],
  model='gemma3:12b', # Podemos cambiar por el modelo que estemos usando, que segun conversamos podria ser gemma3:12b
  format=Patient.model_json_schema(),
)

paciente = Patient.model_validate_json(response.message.content)
print(paciente)

In [ ]:
# Registrar paciente en base de datos
resultado_paciente = coleccion_pacientes.insert_one(paciente.model_dump())
print("ID insertado:", resultado_paciente.inserted_id)

## Extraer informacion de HC

In [ ]:
import re

def extraer_y_eliminar_bloque_historico(texto):
    # Buscar el bloque que comienza con HISTÓRICO DE ATENCIONES y termina en el siguiente encabezado o el final
    match = re.search(r'## HISTÓRICO DE ATENCIONES(.*?)(?=^## |\Z)', texto, re.DOTALL | re.MULTILINE)

    if not match:
        return None, texto  # No se encontró el bloque

    bloque = match.group(0)       # Incluye el título y el contenido
    contenido = match.group(1).strip()  # Solo el contenido debajo del título

    texto_sin_bloque = texto.replace(bloque, '').strip()

    return contenido, texto_sin_bloque

historico_atentionces, texto_limpio = extraer_y_eliminar_bloque_historico(texto_limpio)

In [ ]:
print(historico_atentionces)

In [ ]:
# Separacion de citas medicas

import re
citas_medicas = texto_limpio.split("## NOTAS MÉDICAS")
citas_medicas = [c for c in citas_medicas if c.strip() != ""]

# Expresión regular que busca desde el principio del string
patron = r'Fecha:\s*(\d{2}/\d{2}/\d{4} \d{2}:\d{2})'

atenciones = []

i = 1
for texto in citas_medicas:
  print(f"Procesando historia {i}")
  atencion = {
      "fecha_hora": "",
      "resumen": "",
      "texto": ""
  }
  match = re.match(patron, texto)
  if match:
    atencion["fecha_hora"] = match.group(1)


  resumen = ""

  for j in range(0, len(texto), 5000):
      bloque = texto[j:j+5000]

      response = client.chat(
          model='gemma3:12b',
          messages=[{
              'role': 'user',
              'content': f"Resume la informacion sobre la atencion medica del paciente destacando los datos importantes. No incluyas texto adicional, solo genera el resumen sencillo en un parrafo, no es necesario que especifiques detalles. \n {bloque}",
          }],
          options={
              "temperature": 0.0
          }
      )

      resumen+=response.message.content + " "

  atencion["resumen"] = resumen

  atencion["texto"] = texto

  atenciones.append(atencion.copy())
  i += 1


# registra contenido de historia clinica del paciente
hc = {
    "id_paciente": str(resultado_paciente.inserted_id),
    "historico_atenciones": historico_atentionces,
    "atenciones": atenciones
}

resultado_hc = coleccion_hc.insert_one(hc)
print("ID insertado:", resultado_hc.inserted_id)

In [ ]:
for atencion in atenciones:
  print(atencion["resumen"])
  print('_____________________________')

## Registrando en base de datos vectorial (EN PROGRESO)
### Se evaluara si sera necesario

In [ ]:
# Nuevas dependencias
!pip install chromadb
!pip install -qU langchain-text-splitters

In [ ]:
# Obtencion de texto de HC
from bson import ObjectId

print("Buscando HC: ", resultado_hc.inserted_id)
# Buscar por _id
hc = coleccion_hc.find_one({"_id": ObjectId("68055d4521ff68485c8a293b")})

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# Particionando texto para insertar en bd vectorial
documentos = []
metadatas = []

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

for cm in hc["atenciones"]:
    metadata = {
        "fecha_hora": cm['fecha_hora'],
    }

    texts = text_splitter.create_documents([cm["texto"]])
    nuevos_documentos = [e.page_content for e in texts]
    nuevos_metadatas = [metadata for _ in range(0, len(nuevos_documentos))]

    documentos.extend(nuevos_documentos)
    metadatas.extend(nuevos_metadatas)

ids = [f"{str(hc["_id"])}_{i}" for i in range(0, len(documentos))]

In [6]:
from chromadb import HttpClient
import chromadb.utils.embedding_functions as embedding_functions

chromadb_client = HttpClient()

embed_model = "all-minilm" # Se puede utilizar otro modelo. Incluso podriamos utilizar el gemma3:12b, pero por limitaciones de mi maquina, uso este modelo pequeno. Usaria los embeddings del gemma3:4b, pero no tiene habilitada esa funcion
ollama_ef = embedding_functions.OllamaEmbeddingFunction(
    url="https://300a-2a0c-5a85-d50a-4600-8951-4200-857d-7ed3.ngrok-free.app",
    model_name=embed_model,
)

In [ ]:
collection = chromadb_client.create_collection(
        name=str(hc["_id"]),
        embedding_function=ollama_ef,
        metadata={"hnsw:space": "cosine"} # l2 is the default
)

collection.add(documents=documentos, metadatas=metadatas, ids=ids)

In [ ]:
# Descomentar en caso se desee ELIMINAR la coleccion
# chromadb_client.delete_collection(hc["_id"])

In [9]:
# Funcion para consultar la coleccion de ChromaDB
def query_chromadb(query_text, fecha_hora, n_results=1):
    results = collection.query(
        query_texts=[query_text],
        where={"fecha_hora": fecha_hora},
        n_results=n_results
    )
    return results["documents"], results["metadatas"]


In [ ]:
query = "exámenes físicos"  # Change the query as needed
response = query_chromadb(query, fecha_hora="21/06/2024 12:41", n_results=1)
print("######## Response from CHROMADB ########\n", response)